# TEXT MINING for BEGINNER
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

## 실전 텍스트 분석하기: LDA토픽모델링, 감성분석
- Python을 활용해 텍스트 데이터를 분석하는 여러가지 방법에 대해 다룹니다.

---

### 1. LDA토픽모델링: 단어를 문서에 포함된 주제단위로 군집화하기

---

#### 1.1. 샘플 데이터 불러오기

---

In [1]:
# 기사본문을 저장할 리스트를 생성합니다.
article_list = []

# 기사가 저장된 파일은 data 폴더 안에 저장되어 있습니다.
file_path = "/Users/ijiseon/Desktop/ESAA-OB/article_sample.txt"

# 기사가 저장된 파일을 불러옵니다.
f = open(file_path, encoding="utf-8")
for line in f:
    col = line.strip().split("\t")
    #group = col[0]
    #source = col[1]
    #datetime = col[2]
    #title = col[3]
    # 기사의 본문만 가져와 리스트에 저장합니다.
    content = col[4]
    article_list.append(content)
f.close()

print("총 기사 수 :", len(article_list))

총 기사 수 : 100


In [2]:
print("기사샘플 :")
print(article_list[3][:1000])

기사샘플 :
10년 역사 10대 뉴스로 정리(지디넷코리아=임유경 기자)2008년 9월 리먼 브라더스 파산으로 촉발된 금융위기가 전 세계를 뒤흔들었다. 세계 금융시스템이 붕괴 위기에 놓여 있던 이때, 인터넷 세상 한 켠에선 새로운 금융 시스템이 조용하게 작동을 시작했다. 10년 전 1월 3일 오후 6시 15분. 은행이나 정부의 개입이 필요 없는 개인 간 전자화폐 시스템 비트코인 네트워크의 첫 번째 블록이 탄생한 순간이다.이날 사토시 나카모토라는 가명의 인물은 앞서 공개한 논문을 통해 제시한 아이디어를 약 한달만에 구현하는 데 성공했다. 기념비적인 첫 번째 블록에는 "재무장관, 은행에 두 번째 구제금융 제공 임박”이라는 문구를 새겼다. 이날 영국 일간지 <타임스>의 1면 헤드라인으로, 비트코인 네트워크가 기존 금융 시스템에 대안적인 성격을 띤다는 정체성을 분명하게 드러냈다.이후 비트코인은 굴곡진 10년의 세월을 보냈다. 2011년 초까지 1달러의 가치도 인정 받지 못했다. 비트코인에 대한 관심도 암호학 커뮤니티 내 개발자들이 재미로 채굴(트랜잭션을 블록단위로 묶어 처리하는 작업)하고 비트코인을 받는 수준에 그쳤다. 이후 관심을 늘었지만 투기기 불법적인 요소 결합돼 버블이니 검은 돈이니 하는 오명을 얻기도 했다.2009년 1월 3일 비트코인 첫 번째 블록인 제네시스 블록이 생성됐다.여전비 비트코인은 실험대 위에 있다. 가격이 출렁일 때마다 "비트코인은 죽었다"는 제목의 기사가 쏟아져 나온다. 지난해 언론에서 내린 사망선고는 거의 100번에 이른다. (☞관련링크)분명한 것은 비트코인의 잠재력을 믿는 사람들이 늘어나고 있다는 점이다. 이제 전 세계 모든 주요 기업들이 비트코인 기반 기술인 블록체인의 혁신성에 주목하고 기존 비즈니스에 적용하기 위해 연구에 나섰다.비트코인 맥시멀리스트로 알려진 지미 송은 최근 자신의 블로그에 비트코인이 가진 특성 중 탈중앙성(정부나 은행의 개입이 없는 상태)에 주목해야 한다고 강조하며 "기업가들은 비트코인 내에서 자신의 돈과 노력을 가지고

#### 1.2. 기사 문장을 형태소 단위로 분리하기

---

In [3]:
%env JAVA_HOME=/usr/local/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home

env: JAVA_HOME=/usr/local/opt/openjdk@11/libexec/openjdk.jdk/Contents/Home


In [4]:
# 꼬꼬마 형태소 분석기를 불러옵니다.
from konlpy.tag import Kkma
kkma = Kkma()

text = "꼬꼬마 형태소 분석기를 불러옵니다."
print(kkma.pos(text))

[('꼬꼬마', 'NNG'), ('형태소', 'NNG'), ('분석기', 'NNG'), ('를', 'JKO'), ('불러오', 'VV'), ('ㅂ니다', 'EFN'), ('.', 'SF')]


In [5]:
# 형태소분석 후 기사단위의 형태소 리스트를 저장할 리스트를 생성합니다.
tag_sentence_list = []

# 불용어를 리스트를 정의합니다.
stop_list = ["카페", "카페규정"]

# 형태소 중 꼭 포함해야하는 형태소 태그를 리스트로 생성합니다.
pos_list = ["NNG", "NNP"]

now = 0
for article in article_list:
    now += 1
    print(now, end="\r")
    sentence_list = kkma.sentences(article)
    tag_sentence = []
    for sentence in sentence_list:
        tag_list = kkma.pos(sentence)
        for word, pos in tag_list:
            if pos in pos_list and word not in stop_list and len(word) > 1:
                tag_sentence.append(word)
    tag_sentence_list.append(tag_sentence)

#### 1.3. LDA토픽모델링 수행하기

---

In [6]:
# 텍스트 분석을 위한 다양한 기능을 제공하는 gensim 패키지를 설치합니다.

!pip install gensim

In [7]:
# gensim 패키지를 불러옵니다.
import gensim
from gensim import corpora

# LDA토픽모델링 수행중에는 진행중인 상황에 대한 로그가 출력됩니다.
# 아래 7~9번째 줄을 주석처리하면 로그를 확인할 수 있습니다.
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

In [9]:
# 추출할 토픽의 개수를 지정합니다.
my_num_topic = 3

# 각 토픽에 포함할 키워드(단어) 개수를 지정합니다.
my_num_keyword = 10

In [10]:
# LDA토픽모델링을 수행합니다.

dictionary = corpora.Dictionary(tag_sentence_list)
corpus = [dictionary.doc2bow(text) for text in tag_sentence_list]

In [13]:
dictionary

In [11]:
corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 2),
  (4, 3),
  (5, 1),
  (6, 1),
  (7, 2),
  (8, 8),
  (9, 2),
  (10, 1),
  (11, 3),
  (12, 2),
  (13, 6),
  (14, 9),
  (15, 1),
  (16, 1),
  (17, 2),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 2),
  (25, 1),
  (26, 3),
  (27, 5),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 3),
  (38, 1),
  (39, 1),
  (40, 5),
  (41, 3),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 2),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 3),
  (59, 1),
  (60, 3),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 2),
  (65, 4),
  (66, 1),
  (67, 1),
  (68, 14),
  (69, 1),
  (70, 2),
  (71, 1),
  (72, 2),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 2),
  (77, 1),
  (78, 1),
  (79, 2),
  (80, 2),
  (81, 1),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 2),
  (86, 1),
  (87, 5),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1

In [14]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=my_num_topic, id2word = dictionary, passes=10)

In [16]:
# 추출된 토픽과 토픽에 포함된 키워드를 저장할 리스트를 생성합니다.
topic_list = []

# 추출된 토픽을 리스트에 저장합니다.
for num in range(my_num_topic):
    topic_list.append(ldamodel.show_topic(num, my_num_keyword ))

In [17]:
topic_list

[[('금융', 0.049120657),
  ('부동산', 0.040160567),
  ('금리', 0.03879723),
  ('대출', 0.02443075),
  ('은행', 0.019721512),
  ('기준', 0.01438058),
  ('신탁', 0.010352521),
  ('그림자', 0.010344678),
  ('투자', 0.009007518),
  ('시장', 0.008533445)],
 [('비트', 0.058801472),
  ('화폐', 0.021032779),
  ('암호', 0.016877422),
  ('거래', 0.011786559),
  ('달러', 0.010319084),
  ('블록', 0.009131999),
  ('금융', 0.009086059),
  ('체인', 0.009041243),
  ('인은', 0.008496547),
  ('시장', 0.0080155255)],
 [('금리', 0.054119274),
  ('대출', 0.034668516),
  ('금융', 0.029693052),
  ('은행', 0.01842516),
  ('자산', 0.0149236275),
  ('평균', 0.012236343),
  ('기준', 0.012183159),
  ('최고', 0.011683498),
  ('신용', 0.009768595),
  ('디지털', 0.009691073)]]

In [18]:
# 추출된 토픽을 출력합니다.
for index in range(len(topic_list)):
    print("# 주제", index + 1)
    for topic in topic_list[index]:
        print("\t", topic[0].split("/")[0])

# 주제 1
	 금융
	 부동산
	 금리
	 대출
	 은행
	 기준
	 신탁
	 그림자
	 투자
	 시장
# 주제 2
	 비트
	 화폐
	 암호
	 거래
	 달러
	 블록
	 금융
	 체인
	 인은
	 시장
# 주제 3
	 금리
	 대출
	 금융
	 은행
	 자산
	 평균
	 기준
	 최고
	 신용
	 디지털


#### 1.4. LDA토픽모델링 결과 시각화하기

---

In [8]:
# LDA토픽모델링 결과를 시각화하는 pyLDAvis 패키지를 설치합니다.

!pip install pyLDAvis

  Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 4.6 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
islp 0.3.21 requires numpy<1.25,>=1.7.1, but you have numpy 1.26.4 which is incompatible.
islp 0.3.21 requires pandas<=1.9,>=0.20, but you have pandas 2.2.2 which is incompatible.
pytorch-lightning 2.1.0 requires torch>=1.12.0, but you have torch 1.10.1 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.
tor

In [21]:
# pyLDAvis 패키지를 불러옵니다.
import pyLDAvis.gensim

# LDA토픽모델링 시각화 결과를 출력합니다.
pyLDAvis.enable_notebook()
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)

# Intertopic Distance Map의 각 주제가 겹치지 않고 잘 구분되면 적절한 토픽개수라고 볼 수 있습니다.
# 그래프의 분포를 확인하며 토픽의 개수를 조정합니다.
# LDA토픽모델링은 분석 수행시마다 매번 결과가 달라집니다.

### 2. 감성분석: 문장의 감성수준을 수치로 표현하기

---

#### 2.1. 샘플 데이터 불러오기

---

In [22]:
# 영화리뷰을 저장할 리스트를 생성합니다.
review_list = []

# 본 자료에서 제공하는 리뷰는 2019년 개봉작 "엑스맨: 다크 피닉스"의 리뷰입니다.
file_path = "/Users/ijiseon/Desktop/ESAA-OB/review_sample.txt"

# 리뷰가 저장된 파일을 불러옵니다.
f = open(file_path, encoding="utf-8")
for line in f:
    review_text = line.strip()
    review_list.append(review_text)
f.close()

print("총 리뷰 수 :", len(review_list))

총 리뷰 수 : 1000


In [23]:
print("리뷰샘플 :")
print(review_list[10])

리뷰샘플 :
이게 마지막이라고?? 정말?? 이렇게 끝낸다고??


#### 2.2. 리뷰 문장을 형태소 단위로 분리하기

---

In [24]:
# 꼬꼬마 형태소 분석기를 불러옵니다.
from konlpy.tag import Kkma
kkma = Kkma()

text = "꼬꼬마 형태소 분석기를 불러옵니다."
print(kkma.pos(text))

[('꼬꼬마', 'NNG'), ('형태소', 'NNG'), ('분석기', 'NNG'), ('를', 'JKO'), ('불러오', 'VV'), ('ㅂ니다', 'EFN'), ('.', 'SF')]


In [25]:
# 형태소분석 후 기사단위의 형태소 리스트를 저장할 리스트를 생성합니다.
tag_sentence_list = []

# 불용어를 리스트를 정의합니다.
stop_list = ["카페", "카페규정"]

now = 0
for review in review_list:
    now += 1
    print(now, end="\r")
    tag_list = kkma.pos(review)
    tag_sentence = []
    for word, pos in tag_list:
        tag_sentence.append(word + "/" + pos)
    tag_sentence_list.append(tag_sentence)

In [26]:
print("리뷰 형태소분석 샘플 :")
print("- 원문 :", review_list[10])
print("- 결과 :", tag_sentence_list[10])

리뷰 형태소분석 샘플 :
- 원문 : 이게 마지막이라고?? 정말?? 이렇게 끝낸다고??
- 결과 : ['이것/NP', '이/JKS', '마지막/NNG', '이/VCP', '라고/ECD', '??/SW', '정말/MAG', '??/SW', '이렇/VA', '게/ECD', '끝내/VV', 'ㄴ다고/ECE', '??/SW']


#### 2.3. 감성사전 불러오기

---

In [ ]:
# 본 자료에서는 KOSAC(Korean Sentiment Analysis Corpus)이라는 공개 감성사전을 활용합니다.
# 감성사전은 아래 링크에서 다운로드 가능합니다.
# 다운로드한 파일 준 polarity.csv 파일을 활용합니다.

감성사전 다운로드 URL : http://word.snu.ac.kr/kosac/lexicon.php

In [27]:
# KOSAC 감성사전의 내용을 살펴봅니다.
f = open("/Users/ijiseon/Desktop/ESAA-OB/polarity.csv", encoding="utf-8")
for line in f.read().split("\n")[:10]:
    line = line.strip().split(",")
    print(line)
f.close()

['ngram', 'freq', 'COMP', 'NEG', 'NEUT', 'None', 'POS', 'max.value', 'max.prop']
['가*/JKS', '1', '0', '0', '0', '0', '1', 'POS', '1']
['가*/JKS;있/VV', '1', '0', '0', '0', '0', '1', 'POS', '1']
['가*/JKS;있/VV;었/EP', '1', '0', '0', '0', '0', '1', 'POS', '1']
['가*/VV', '3', '0', '0', '0', '0', '1', 'POS', '1']
['가*/VV;ㄴ다*/EF', '1', '0', '0', '0', '0', '1', 'POS', '1']
['가/JKC', '17', '0', '0.470588235', '0.235294118', '0', '0.294117647', 'NEG', '0.470588235']
['가/JKC;되/VV', '11', '0', '0.363636364', '0.272727273', '0', '0.363636364', 'NEG', '0.363636364']
['가/JKC;되/VV;ㄴ/ETM', '2', '0', '0', '1', '0', '0', 'NEUT', '1']
['가/JKC;되/VV;ㄹ/ETM', '1', '0', '0', '1', '0', '0', 'NEUT', '1']


In [28]:
# KOSAC 감성사전을 불러와 딕셔너리에 저장합니다.
# 원본 사전에 세미콜론(;)으로 구분된 형태소를 띄어쓰기 단위로 변형하여 딕셔너리의 키(key)값으로 활용합니다.

# 사전을 저장할 딕셔너리를 생성합니다.
sentiment = {}

f = open("/Users/ijiseon/Desktop/ESAA-OB/polarity.csv", encoding="utf-8")
for line in f:
    col = line.strip().split(",")
    pos = col[0]
    polarity = col[7]
    score = col[8]
    # 극성이 POS인 경우 +, NEG인 경우 -를 곱해 스코어(score)를 계산합니다.
    if polarity == "POS":
        score = float(score)
    elif polarity == "NEG":
        score = -float(score)
    else:
        # 긍정 또는 부정인 경우만 취급합니다.
        continue
    # 세미콜론(;)을 띄어쓰기로 바꿔 딕셔너리에 스코어와 함께 저장합니다.
    sentiment[pos.replace(";", " ")] = score

f.close()

In [29]:
# 생성한 감성사전 딕셔너리 샘플을 출력합니다.

# 긍정
print("(긍정) 솜씨/NNG 는/JX 능숙/XR :", sentiment["솜씨/NNG 는/JX 능숙/XR"])
print("(긍정) 자연/NNG 스럽/XSA :", sentiment["자연/NNG 스럽/XSA"])
print("(긍정) 획일/XR 성/XSN 의/JKG :", sentiment["획일/XR 성/XSN 의/JKG"])
print("(긍정) 훨씬/MAG 안전/NNG :", sentiment["훨씬/MAG 안전/NNG"])
# 부정
print("(부정) 뒷맛/NNG 이/JKS 씁쓸/XR :", sentiment["뒷맛/NNG 이/JKS 씁쓸/XR"])
print("(부정) 손해/NNG 배상/NNG 을/JKO :", sentiment["손해/NNG 배상/NNG 을/JKO"])
print("(부정) 에서/JKB 욕/NNG 튀어나오/VV :", sentiment["에서/JKB 욕/NNG 튀어나오/VV"])
print("(부정) 유달리/MAG 난해/XR :", sentiment["유달리/MAG 난해/XR"])

(긍정) 솜씨/NNG 는/JX 능숙/XR : 1.0
(긍정) 자연/NNG 스럽/XSA : 1.0
(긍정) 획일/XR 성/XSN 의/JKG : 1.0
(긍정) 훨씬/MAG 안전/NNG : 1.0
(부정) 뒷맛/NNG 이/JKS 씁쓸/XR : -1.0
(부정) 손해/NNG 배상/NNG 을/JKO : -1.0
(부정) 에서/JKB 욕/NNG 튀어나오/VV : -1.0
(부정) 유달리/MAG 난해/XR : -1.0


#### 2.4. 리뷰 감성스코어 계산하기

---

In [30]:
# 감성분석 결과를 긍정과 부정 리뷰로 각각 저장할 리스트를 생성합니다.
positive_sentence = []
negative_sentence = []

for tag_sentence, review in zip(tag_sentence_list, review_list):
    # 스코어를 0점으로 초기화합니다.
    score = 0.0
    # 형태소분석된 문장 안에서 최대 7개까지 결합된 형태소가 감성사전 안에 있는지 확인합니다.
    for i in range(len(tag_sentence)):
        max_n = 7
        for n in range(max_n):
            ngram = ""
            for j in range(len(tag_sentence)-n):
                ngram += " " + tag_sentence[j]
            ngram = ngram.strip()
            if ngram in sentiment.keys():
                score += sentiment[ngram]
    # 감성스코어로 긍정 또는 부정을 판단하기위한 임계값을 지정합니다.
    limit = 0
    if score > limit:
        positive_sentence.append(["POSITIVE", str(review)])
    elif score < -limit:
        negative_sentence.append(["NEGATIVE", str(review)])

# 감성분석 결과를 감성스코어를 기준으로 정렬합니다.
positive_sentence.sort(reverse=True)
negative_sentence.sort()

#### 2.5. 리뷰 감성분석 결과 확인하기
- 본 자료에서 사용하는 KOSAC 감성사전은 표준어를 바탕으로 제작되었습니다.
- 온라인 리뷰의 특성상 문법의 오류가 굉장히 많기 때문에 사전과의 매칭률이 낮을 수 있습니다.
- 또한 KOSAC 감성사전의 형태소태깅과 본 자료에서 사용한 꼬꼬마 형태소분석기의 형태소태깅 결과의 차이도 영향을 미칩니다.
- 본 자료의 결과가 모두 맞는 것은 아니며 사전기반의 감성분석을 수행하는 방법에 대한 이해를 위해 사용해주세요.
- 직접 분석하려는 텍스트의 도메인에 맞는 사전을 추가하여 정확도를 높힐 수 있습니다.

---

In [31]:
print("긍정판별 문장 개수 :", len(positive_sentence))
positive_sentence[:10]

긍정판별 문장 개수 : 27


[['POSITIVE', '흥미진진 마지막액션씬최고'],
 ['POSITIVE', '하루빨리 통합마블 되기를'],
 ['POSITIVE', '최고최고최고최고최고!'],
 ['POSITIVE', '최고ㅜㅜ 레이븐ㅠㅠㅠ'],
 ['POSITIVE', '장난합니까 마지막 영환데'],
 ['POSITIVE', '은혜를 모르는 진 그레이..'],
 ['POSITIVE', '영화보는데 기빨린다 후하'],
 ['POSITIVE', '영화를 만들다 말았어...'],
 ['POSITIVE', '아주 재밌습니다 후회없음'],
 ['POSITIVE', '생각보다 많이 괜찮은 영화']]

In [32]:
print("부정판별 문장 개수 :", len(negative_sentence))
negative_sentence[:10]

부정판별 문장 개수 : 13


[['NEGATIVE', '계속 생각 나는 영화. ㅠㅠ'],
 ['NEGATIVE', '그냥 개똥보다 못한 영화'],
 ['NEGATIVE', '난재밌었어요....'],
 ['NEGATIVE', '내 개인 평점 7.2..;;'],
 ['NEGATIVE', '말이 필요없습니다. 재미있어요'],
 ['NEGATIVE', '스토리 너무 늘어지는듯'],
 ['NEGATIVE', '스토리가 왜이리 이상하냐'],
 ['NEGATIVE', '아쉬움가득안고떠납니다'],
 ['NEGATIVE', '아쉽지만 행복했습니다'],
 ['NEGATIVE', '왜 엑스우먼이 아닌지ㅎ']]